<a href="https://colab.research.google.com/github/ahmedhisham73/Googlework/blob/master/update_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
import json
import requests
import datetime
 
def cases_list(country = "egypt" , limit=10000):
    URL = "https://disease.sh/v2/historical/{}?lastdays={}"
    response=requests.get(URL.format(country,limit))
    response = response.json()
    cases_dict = response["timeline"]["cases"]
    cases_list = sorted(list(cases_dict.values()))
    dates_list = list(cases_dict.keys())
    last_date = dates_list[-1]
    return last_date , cases_list
 
def update_predictions(predictions , last_date , URL ="" ):
    data = {}
    date = datetime.datetime.strptime(last_date, '%m/%d/%y')
    for i in range(len(predictions)):
        date += datetime.timedelta(days=1)
        data[str(date.date())]=predictions[i]
    requests.post(URL, json=json.dumps(data))
    return data
 
 
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
 
def get_predictions(cases_list):
    raw_seq = cases_list
# choose a number of time steps
    n_steps = 5
# split into samples
    X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
    n_features = 1
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    model = Sequential()
    model.add(Bidirectional(LSTM(2000, activation='relu'), input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
# fit model
    model.fit(X, y, epochs=400)
# demonstrate prediction
    x_input = array(cases_list[-5:])
    x_input = x_input.reshape((1, n_steps, n_features))
    #model.save_weights('covid_Egypt1.h5')
    predicitions = model.predict(x_input, verbose=0)
    predicitions=int(predicitions)
    #print(predicitions)
    return [predicitions]
 
last_date , cases = cases_list()
predictions = get_predictions(cases)
#print(update_predictions(predictions , last_date , "https://prediction-covid.herokuapp.com/update"))
print(update_predictions(predictions , last_date , "https://predictions-db.herokuapp.com/update"))

Epoch 1/400
128/128 [==============================] - 1s 6ms/step - loss: 32531262.5635
Epoch 2/400
128/128 [==============================] - 0s 2ms/step - loss: 1804664.5781
Epoch 3/400
128/128 [==============================] - 0s 2ms/step - loss: 622482.5918
Epoch 4/400
128/128 [==============================] - 0s 2ms/step - loss: 31355.2012
Epoch 5/400
128/128 [==============================] - 0s 2ms/step - loss: 166200.5198
Epoch 6/400
128/128 [==============================] - 0s 2ms/step - loss: 83790.4185
Epoch 7/400
128/128 [==============================] - 0s 2ms/step - loss: 175654.2051
Epoch 8/400
128/128 [==============================] - 0s 2ms/step - loss: 73583.6260
Epoch 9/400
128/128 [==============================] - 0s 2ms/step - loss: 56096.8362
Epoch 10/400
128/128 [==============================] - 0s 2ms/step - loss: 63111.7393
Epoch 11/400
128/128 [==============================] - 0s 2ms/step - loss: 287887.7534
Epoch 12/400
128/128 [=====================